In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
from metrics import accuracy

from sklearn import datasets
from sklearn.model_selection import train_test_split

# <span style="color:#F08E21">Support Vector Machine (SVM)</span>

Is a versatile and powerful ML model and can perform both linear and non-linear classification, regression or outlier detection.\
They can be particularly well-suited to classification for complex but small to medium sized datasets.\
SVM is sensitive to feature scales -> Scaling can be of benefit.

### <span style="color:#217AB8">Linear Classification with SVM</span>
Set the boundary between classes so as to keep the largest margin between them, or placing the widest possible street between the classes. -> large margin classification.\
The decision boundary is determined/supported by the instances located on the edge of the street (named the support vectors). If you added more data that is beyond the street it would change nothing about the decision boundary.


**&emsp;Soft vs Hard Margin Classification**\
&emsp; Hard margin: If all instances must be off the street and on the right side.\
&emsp; Downsides: Only works with linearly seperable data and is very sensitive to outliers. 

&emsp; Soft-margin: Find a balance between keeping the street as wide as possible and limit the margin violations.\
&emsp; A slightly more flexible model that may generalize better.

&emsp; By controlling the width of the street eg. c variable the number of margin violations can be increased or decreased. (eg. scikit-learn high c lower margin violation)

### <span style="color:#217AB8">Non-Linear Classification with SVM</span>
When the instances can not be separated by a linear decision boundary SVM makes use of non-linear methods.

**&emsp;Polynomial Kernel**\
&emsp;Using the technique called the kernel trick it possible to get the same results as adding many polynomial features to your dataset, without actually having to add them.\
&emsp;Setting the polynomial to high can lead to overfitting, while setting it too low can lead to underfitting (eg. attempting linear boundaries with non-linear classes)

**&emsp;Similarity Features**\
&emsp;Add features computed using a similarity function that measures how similar an instance is to a landmark.\
&emsp;An example of such a similarity function could be the Gaussian Radial Bias Function (RBF) which is a bell shaped function where 0 means the instance is far away from the landmark and 1 means it is right on top.\
&emsp;Applying this function and using it as the features to describe the data set makes the instances linearly separable. (selecting the landmarks close to each instance makes it more likely to become linearly separable.\
&emsp;However this explodes the dataset to have as many features as samples)\
&emsp;Again the Kernel trick can help you achieve the same complexity without adding so many new features.

### <span style="color:#217AB8">Regression with SVM</span>

